In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm
from collections import deque
from dataclasses import dataclass
from itertools import combinations
import time
import math

In [2]:
# seqDist simple
def seqDist(s1, s2):
    return sum([1 if b1 != b2 else 0 for b1, b2 in zip(s1, s2)])

In [3]:
# seqDist with a max_dist
def seqDist(s1, s2, max_dist):
    dist = 0
    for b1, b2 in zip(s1, s2):
        if dist > max_dist:
            return len(s1)
        else:
            if b1 != b2:
                dist += 1
    return dist

In [4]:
def mutateSeq(S, mut):
    pos, mut_base = mut           
    
    if pos >= len(S):
        raise Exception("Position out of range") 
    
    if pos == 0:
        return mut_base + S[1:]
    elif pos == len(S)-1:
        return S[:len(S)-1] + mut_base
    else:
        return S[:pos] + mut_base + S[pos+1:]

In [5]:
def makeMutations(S, mut):
    Sx1 = S
    for i in range(0, len(mut[0])):
        Sx2 = mutateSeq(Sx1, (mut[0][i], mut[1][1][i]))
        Sx1 = Sx2
    return Sx1

In [6]:
def combineMut(mut):
    
    pos_combined = ()
    ref_base_combined = ''
    mut_base_combined = ''
    
    for m in mut:
        pos_combined += m[0]
        ref_base_combined += m[1][0]
        mut_base_combined += m[1][1]
        
    return [pos_combined, (ref_base_combined, mut_base_combined)]

In [7]:
@dataclass
class BfsNode:
    seq: str
    depth: int

def BFS(start_seq_seq, seqset, mutations, max_depth, s2cpm):

    blacklist = set()
    explored_seqs = {x : [] for x in range(0, max_depth+1)}
    
    queue = deque([BfsNode(seq=start_seq_seq, depth=0)])
    explored_seqs[0].append(start_seq_seq)
    
    while queue:
        current_node = queue.popleft()
        if current_node.depth < max_depth:
            mutants_batch = [makeMutations(current_node.seq, mut) for mut in mutations if ''.join([current_node.seq[x] for x in mut[0]]) == mut[1][0]]
            mutants_batch = [m for m in mutants_batch if (m not in blacklist) and (m in seqset)]
            for mutant in mutants_batch:
                next_node = BfsNode(seq=mutant, depth=current_node.depth+1)
                queue.append(next_node)
                explored_seqs[next_node.depth].append(next_node.seq)
                blacklist.add(mutant)
    return explored_seqs

In [8]:
# Load the entire dataset
df = pd.read_csv('/home/jardic/Documents/projects/jk/datasets/datasets_prepped/strc_km.csv', usecols=['varseq', 'cpm'])
seq_2_cpm = {s : c for s, c in zip(df['varseq'], df['cpm'])}

In [9]:
seqlen = len(df.iloc[0]['varseq'])
aurora2 = 'AGACATGTTTTGTAAATATGTTGT'

In [10]:
# Load the possible mutations
with open('../prep/allowed_mutations_permuations.pkl', mode='rb') as mf:
    mutations = pickle.load(mf)

# Create combinations of mutations
mutation_combinations_1 = [mut_comb for mut_comb in list(combinations(mutations, r=1))]
mutation_combinations_2 = [mut_comb for mut_comb in list(combinations(mutations, r=2)) if len(set([mut_comb[0][0], mut_comb[1][0]])) == 2]
mutation_combinations_3 = [mut_comb for mut_comb in list(combinations(mutations, r=3)) if len(set([mut_comb[0][0], mut_comb[1][0], mut_comb[2][0]])) == 3]

# Combine the mutation combinations so that they can be applied with the function makeMutations
mutations_combined_1 = [combineMut(mut_c) for mut_c in mutation_combinations_1]
mutations_combined_2 = [combineMut(mut_c) for mut_c in mutation_combinations_2]
mutations_combined_3 = [combineMut(mut_c) for mut_c in mutation_combinations_3]

mutations_combined_1 = [m for m in mutations_combined_1 if ''.join([aurora2[x] for x in m[0]]) == m[1][0]]
mutations_combined_2 = [m for m in mutations_combined_2 if ''.join([aurora2[x] for x in m[0]]) == m[1][0]]
mutations_combined_3 = [m for m in mutations_combined_3 if ''.join([aurora2[x] for x in m[0]]) == m[1][0]]

# Print the number of mutations for each level
print(len(mutations_combined_1))
print(len(mutations_combined_2))
print(len(mutations_combined_3))

35
556
5338


In [11]:
ref_wt_seq = aurora2
ref_wt_cpm = seq_2_cpm[ref_wt_seq]

Let me define four variables:
- ref_wt_cpm: Aurora2 cpm
- ref_mut_cpm: The cpm of the sequence we get when we apply a given mutation on Aurora2
- probe_wt_cpm: The cpm of a probe sequence (any sequence which is not Aurora2 and that the mutation can be applied to)
- probe_mut_cpm: The cpm of the probe sequence mutated

For each mutation:
- Get ref_wt_cpm (this is the cpm of Aurora2)
- Apply mutation mut on Aurora2.
- Get the ref_mut_cpm of this mutant of Aurora2
- Get a list of 'relevant sequences'. Since each mutation in my mutations list is defined 'from what to what', there are only some sequences on which a given mutation can be applied on.
- Now get the Aurora2 background sequence, this I get by concatenating all positions of Aurora2 except for those positions which are involved in the mutation (this will be 1 or 2 positions for 'singles', 2 or 3 or 4 for 'doubles' and 3 or 4 or 5, or 6 for 'triples').
- Get the background sequences from the list of relevant sequences.
- Compute the distances of each of the background distances to aurora2 background sequence.
- Now go down the list of relevant sequences and their background distances and for each, get the probe_wt_cpm, then apply the mutation and get the probe_mut_cpm
- 

In [12]:
results_final = []

for mut in tqdm(mutations_combined_1):

    mutname = '-'.join([str(x) for x in mut[0]]) + '|' + '>'.join([x for x in mut[1]])
    mut_res = []

    # Get the effect on the reference, if possible
    ref_mut_seq = makeMutations(ref_wt_seq, mut)
    ref_mut_cpm = seq_2_cpm.get(ref_mut_seq, None)

    if ref_mut_cpm == None:
        continue # If not, go to the next mutation, nothing left to do
    else:
        ref_mut_effect = ref_mut_cpm / ref_wt_cpm

    # Get the reference background
    ref_background = ''.join([ref_wt_seq[x] for x in range(0, seqlen) if x not in mut[0]])

    # Get only relevant sequences (those, on which this particular mutation can be applied)
    rel_seqs = [s for s in df['varseq'] if ''.join([s[x] for x in mut[0]]) == mut[1][0]]

    # Get backgrounds of the relevant sequences
    background_seqs = [''.join([s[x] for x in range(0, seqlen) if x not in mut[0]]) for s in rel_seqs]

    # Get background distances of the relevant sequences from the reference
    background_dists = [seqDist(b, ref_background, max_dist=10) for b in background_seqs]

    # Iterate over the relevant probe sequences, compute the effect of mutation mut on them and save according to background distance
    for probe_wt_seq, bg_dist in zip(rel_seqs, background_dists):
        probe_wt_cpm = seq_2_cpm[probe_wt_seq]
        probe_mut_seq = makeMutations(probe_wt_seq, mut)
        probe_mut_cpm = seq_2_cpm.get(probe_mut_seq, None)
        if probe_mut_cpm == None:
            continue
        else:
            probe_mut_effect = probe_mut_cpm / probe_wt_cpm
            mut_res.append([bg_dist, math.log(probe_mut_effect / ref_mut_effect, 10)])

    #for d in mut_res.keys():
    #    if len(mut_res[d]) > 1:
    #        results_final.append([mutname, np.array(mut_res[d]).mean(), d])
    df_mut_res = pd.DataFrame(mut_res, columns=['d', 'log_diff_effect'])
    results_final.append(df_mut_res)

100%|███████████████████████████████████████████████████████████████████████████████████| 35/35 [07:13<00:00, 12.38s/it]


In [19]:
results_final_out = [(mut, df_m) for mut, df_m in zip(mutations_combined_1, results_final)]

In [24]:
with open('results_for_plotting.pkl', mode='wb')as f:
    pickle.dump(results_final_out, f)